# PPO（近端策略算法）
[Proximal Policy Optimization Algorithms](https://arxiv.org/abs/1707.06347)



## 基础概念
![强化学习概念](img/rl.png)

- action space: 可选择的动作
  - {$\uparrow, \rightarrow, \downarrow, \leftarrow$} 记为action  {$a_1, a_2, a_3, a_4$}

- Policy: 策略函数，输入state 输出 action 的概率分布
  - $\pi(a_1| s_t ) = 0.1$
  - $\pi(a_2| s_t ) = 0.5$

- Trajectory：轨迹一般用 $\tau$ 表示表示一连串的状态-动作序列。
  - $ \tau \thicksim {s_1, a_2, s_2, a_3, s_5, ...}$
  - 状态转移：
    - 确定： $s_{t + 1} = f(s_t, a_t)$
    - 不确定：$s_{t +1 } = f(\cdot|s_t,a_t)$

- Return：回报，从当前时间点到游戏结束的reward的累计和。
 - 最大最后收益retuern，而非当前的reward

## 采样

$$\mathrm{E}(x)_{x \thicksim p(x)} = \sum _{x} x p(x) \approx \frac{1}{N} x{ _{x_i \thicksim p(x_i)}}$$

目标，训练一个Policy神经网路$\pi$，在所有状态state下，给出相应的action和得到的return
$$
E(R(\tau))_{\tau \thicksim P_{\theta}(\tau)} = \sum_{\tau}R(\tau)P_{\theta}(\tau)\\
$$
$\theta$ 是Policy网络的参数， $\tau$ trajectory也是由神经网络决定的。

我们使用梯度上升的方法最大化return
$$
\begin{aligned}
\nabla E(R(\tau))_{\tau \thicksim P_{\theta}(\tau)} &= \nabla \sum_{\tau}R(\tau)P_{\theta}(\tau)\\
&= \sum_{\tau}R(\tau) \nabla P_{\theta}(\tau)\\
&= \sum_{\tau}R(\tau) \nabla P_{\theta}(\tau) \frac{P_{\theta}(\tau)}{P_{\theta}(\tau)}\\
&= \sum_{\tau}R(\tau) P_{\theta}(\tau) \frac{\nabla P_{\theta}(\tau)}{P_{\theta}(\tau)}\\
&\approx \frac{1}{N} \sum_{n}^{N} R(\tau^n) \frac{\nabla P_{\theta}(\tau^n)}{P_{\theta}(\tau^n)} \quad \text{n是假标号 且} \nabla log f(x) = \frac{\nabla f(x)}{f(x)} \\
&= \frac{1}{N} \sum_{n}^{N} R(\tau^n)\nabla log P_{\theta}(\tau^n) \quad \text{其中} \tau \thicksim P_{\theta}(\tau)\\
&=\frac{1}{N} \sum_{n}^{N} R(\tau^n)\nabla \prod_{t = 1}^{T_n} log P_{\theta}(a_n^t|s_n^t)\quad \text{假设下一状态是由当前状态和动作决定}\\
&= \frac{1}{N} \sum_{n}^{N} R(\tau^n) \sum_{t = 1}^{T_n} \nabla log P_{\theta}(a_n^t|s_n^t) \\
&= \frac{1}{N} \sum_{n}^{N} \sum_{t = 1}^{T_n} R(\tau^n)  \nabla log P_{\theta}(a_n^t|s_n^t) \\
\end{aligned}
$$
从最后的表达式来看是：  
采样得到的轨迹 $\tau^{(n)}$ 就能得到性能目标函数 
$$J(\theta) = E_{\tau \thicksim P}[R(\tau)]$$
对策略参数 $\theta$ 的梯度 $\nabla_{\theta} J(\theta)$  
用这条梯度做策略改进： $$\theta \leftarrow \theta + \nabla_{\theta} J(\theta)$$

## 训练神经网络
$$ 
loss = - \frac{1}{N}\sum_{n}^{N} \sum_{t = 1}^{T_n} R(\tau^n)  \nabla log P_{\theta}(a_n^t|s_n^t) 
$$ 
负号是因为我们要最小化loss

$$ 
$$

![训练](img/on%20policy.png)
经过神经网络训练后softmax得到 $P(a_n^t | s_n^t)$ 的概率，  
接着我们让神经网络玩n场游戏，得到n个trajectory，和对应的reward也即 $R(\tau^n)$  
这样就可以进行batch的训练更新神经网络，这样往复循环。  
但是这延伸出一个问题，我们大部分时间都花在采样上了，训练非常慢。

**首先考虑的是reward的改进**

原式是跟据一整条trajectory的reward来更新概率的，但是在做出一个action后按照经验是影响后续的reward，并且影响更大的是靠近action之后的reward。为此引入一个折扣因子（discount factor, $\gamma$）：
$$R(\tau^n) \rightarrow \sum_{t' = t} ^{T_n} \gamma^{t' - t}r_{t'}^{n} = R^n_t$$
表示尽可能用当前的动作对trajectory的retuen影响。
$$
\frac{1}{N} \sum_{n}^{N} \sum_{t = 1}^{T_n} R(\tau^n)  \nabla log P_{\theta}(a_n^t|s_n^t) = \frac{1}{N} \sum_{n}^{N} \sum_{t = 1}^{T_n} R^n_t  \nabla log P_{\theta}(a_n^t|s_n^t) 
$$


并且我们希望在相对好的局势下增加概率，我们引入actor-critis的baesline：
$$
\begin{aligned}
&\frac{1}{N} \sum_{n}^{N} \sum_{t = 1}^{T_n} R(\tau^n)  \nabla log P_{\theta}(a_n^t|s_n^t) \\
&= \frac{1}{N} \sum_{n}^{N} \sum_{t = 1}^{T_n} R^n_t  \nabla log P_{\theta}(a_n^t|s_n^t) \\
&= \frac{1}{N} \sum_{n}^{N} \sum_{t = 1}^{T_n} (R^n_t - B(s_t^n))  \nabla log P_{\theta}(a_n^t|s_n^t) 
\end{aligned}
$$

![](img/a2c.png)

## More definition

- Action-Value Function  
$R^n_t$每次都是一次随机采样，方差很大训练不稳定
$Q_{\theta}(s, a)$ 在state 下，做出action-a，期望的回报，称为动作价值函数。  

- State-Value FUnction   
$V_{\theta}(s)$ 在state-s下，期望的回报，称为状态价值函数

- Advantage Function  
$A_{\theta} = Q_{\theta}(s, a) -V_{\theta}(s)$ 在state -s下做出action -a，比其他动作能带来多少的优势。


有了优势函数的概念我们将公式优化为：  
$$
\frac{1}{N} \sum_{n}^{N} \sum_{t = 1}^{T_n} A_{\theta}(s_t^n, a_t^n) \nabla log P_{\theta}(a_n^t|s_n^t) 
$$
由 $Q_{\theta}(s, a) = r_t + \gamma V_{\theta}(s_{t+1})$ 带入：
$$
A_{\theta}(s_t^n, a_t^n) = r_t + \gamma V_{\theta}(s_{t+1}) - V_{\theta}(s_t)
$$
这样神经网络简化为只需要拟合状态价值函数（State-Value FUnction）


对状态价值函数也进行采样：
$$V_{\theta}(s_{t+1}) \approx r_{r+1} + \gamma V_{\theta}(s_{t+2})$$
* 贝尔曼方程得来

这样我们可以对(s,a)进行：  
一步采样 $A_{\theta}^{1}(s_t^n, a_t^n) = r_t + \gamma V_{\theta}(s_{t+1}) - V_{\theta}(s_t)$\
两步采样 $A_{\theta}^{2}(s_t^n, a_t^n) = r_t + \gamma r_{t+1}V_{\theta}(s_{t+2}) - V_{\theta}(s_t)$\
...\
全采样 $A_{\theta}^{T}(s_t^n, a_t^n) = r_t + \gamma^2 r_{t+3} + ... +\gamma^T r_T - V_{\theta}(s_t)$\
采样越多方差越多偏差越小，为了简介表示定义：
$$
\delta_t^V =  r_t + \gamma V_{\theta}(s_{t+1}) - V_{\theta}(s_t)
$$
转化为：  
$A_{\theta}^{1}(s_t^n, a_t^n) = \delta^V_t$\
$A_{\theta}^{2}(s_t^n, a_t^n) = \delta^V_t + \gamma\delta^V_{t+1}$\
$A_{\theta}^{2}(s_t^n, a_t^n) = \delta^V_t + \gamma\delta^V_{t+1} + \gamma^2\delta^V_{t+2}$
...


## Generalized Advantage Estimation (GAE)
$$
\begin{align*}
A_{\theta}^{GAE}(s_t, a) &= (1 - \lambda)(A_{\theta}^1 + \lambda \cdot A_{\theta}^2 + \lambda^2 A_{\theta}^3 + \cdots) \\
&= (1 - \lambda)(\delta_t^V + \lambda \cdot (\delta_t^V + \gamma \delta_{t+1}^V) + \lambda^2 (\delta_t^V + \gamma \delta_{t+1}^V + \gamma^2 \delta_{t+2}^V) + \cdots) \\
&= (1 - \lambda)(\delta_t^V (1 + \lambda + \lambda^2 + \cdots) + \gamma \delta_{t+1}^V \cdot (\lambda + \lambda^2 + \cdots) + \cdots) \\
&= (1 - \lambda)(\delta_t^V \frac{1}{1 - \lambda} + \gamma \delta_{t+1}^V \frac{\lambda}{1 - \lambda} + \cdots) \\
&= \sum_{b=0}^{\infty} (\gamma \lambda)^b \delta_{t+b}^V
\end{align*}
$$
例子：
$$
\begin{align*}
\lambda = 0.9: \quad A_{\theta}^{GAE} = 0.1 A_{\theta}^1 + 0.09 A_{\theta}^2 + 0.081 A_{\theta}^3 + \cdots
\end{align*}
$$

最后我们的到三个最终式子：
![values function](img/vlues%20function.png)

通过这个神经网络来拟合价值函数--当前步到结束的衰减加和作为label。

## on-policy and off-policy
- on-policy：学习所用的数据必须由当前正在优化的策略产生。
  - 边采样、边更新；每次更新后旧数据作废	
  - REINFORCE、A2C/A3C
- off-policy：学习所用的数据可以来自任何策略（通常是旧策略或行为策略）。
  - 可把旧经验反复回放（replay buffer）
  - 需要 importance sampling（重要性采样） or repaly buffer（经验放回池）

重要性采样：
$$
\begin{align*}
\mathbb{E}(f(x))_{x \sim p(x)} &= \sum_x f(x) \cdot p(x) \\
&= \sum_x f(x) \cdot p(x) \frac{q(x)}{q(x)} \\
&= \sum_x f(x) \frac{p(x)}{q(x)} \cdot q(x) \\
&= \mathbb{E}\left(f(x) \frac{p(x)}{q(x)}\right)_{x \sim q(x)}
\end{align*}
$$
把难以计算/采样效率低的p分布转换为q分布